# A scanpy workflow for the initial basic processing of scRNA-seq data

Note, this workflow does not have many steps- it is meant only to harmonize the basic upstream matrix processing analyses.

This workflow is for processing 10X single cell transcriptomics data with scanpy. It is based on my own workflows, the [scanpy tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html), and [best practise guidlines](https://github.com/theislab/single-cell-tutorial).

Inputs:
- CellRanger v3 matrix files (filtered)-- barcodes.tsv, genes.tsv, matrix.mtx
- You may edit the input paramter `value`s in `params_lst`

Outputs:
- `.h5ad` output file with the `anndata` object containing the gene processed expression matrix

Note, please install the depenancies with the correct versions via the `environment.yml` file using `conda` 

## User input parameters

In [ ]:
import sys

print(sys.version, sys.executable)
assert sys.version_info[:2] == (3, 7)  # should have been installed via conda

In [ ]:
# TODO: find a better way to specify paramters, this is overly complicated
# Argparse style for notebooks or Mypy way for type checking?


from dataclasses import dataclass
from typing import Callable, Any
from inspect import getsource


@dataclass
class Param:
    """Holds information on a particular user input parameter."""

    name: str
    value: Any
    description: str
    func: Callable

    def __bool__(self):
        """Forbid boolean comparisons."""
        raise NotImplementedError()

    def __eq__(self, other):
        """Forbid equality comparisons"""
        raise NotImplementedError()

    def validate(self):
        """Validate the .value satisfies the .func"""
        if not self.func(self.value):
            _err = (
                f"parameter {self.name}={self.value}, "
                f"should satisfy the function: {getsource(self.func).strip()}"
            )
            raise Exception(_err)

In [ ]:
%load_ext blackcellmagic

Specifying the input directory and output file

In [ ]:
# These values (i.e. 1st element like "test_data") should be edited to the desired settings

params_lst = [
    # File/directory names
    Param(
        "input_dir",
        "test_data",  # Value-- can be edited
        "the directory containing subdirectories with the CellRanger filtered matrix.mtx file",
        lambda x: isinstance(x, str),
    ),
    Param(
        "output_file",
        "results.h5ad",  # Value-- can be edited
        ".h5ad output file name",
        lambda x: isinstance(x, str),
    ),
    Param(
        "log_file",
        sys.stdout,  # Value-- can be edited
        "log file name, sys.stdout for printing",
        lambda x: lambda x: x == sys.stdout or isinstance(x, str),
    ),
    # Other params
    Param(
        "min_genes",
        50,  # Value-- can be edited
        "filter cells with fewer than n genes expressed",
        lambda x: isinstance(x, int),
    ),
    Param(
        "min_cells",
        3,  # Value-- can be edited
        "filter genes expressed in fewer than n cells",
        lambda x: isinstance(x, int),
    ),
    Param(
        "highly_var",
        True,  # Value-- can be edited
        "calculate highly variable genes",
        lambda x: isinstance(x, bool),
    ),
    Param(
        "filter_n",
        False,  # Value-- can be edited
        "filter <4500 counts (arbituary value)",
        lambda x: isinstance(x, bool),
    ),
    Param("filter_mt", False, "filter <5% mitocondrial", lambda x: isinstance(x, bool)),
    Param(
        "correct",
        None,  # Value-- can be edited
        "batch correction method, use None or False for no correction",
        lambda x: x in {"bbknn", "mnn", "combat", False, None},
    ),
    Param(
        "doublet_rem",
        False,  # Value-- can be edited
        "remove doublets with scrublet",
        lambda x: isinstance(x, bool),
    ),
    Param(
        "regress",
        False,  # Value-- can be edited
        "regress out (mostly) unwanted sources of variation; not recomended",
        lambda x: isinstance(x, bool),
    ),
    Param(
        "scale",
        False,  # Value-- can be edited
        "scale data to unit variance and zero mean; not recomended",
        lambda x: isinstance(x, bool),
    ),
    Param(
        "exclude_high",
        False,  # Value-- can be edited
        "exclude very highly expressed genes during normalization",
        lambda x: isinstance(x, bool),
    ),
]

for param in params_lst:
    print(
        f"Name: {param.name}, Value: {param.value},\nDescription: {param.description}\n"
    )

for param in params_lst:
    param.validate()

params = {param.name: param for param in params_lst}

List of marker genes to plot later on the UMAP and other projections, you may edit the list

## Initial configuration and logging

Setup logging and imports

In [ ]:
import os
import glob

import logging
logging.basicConfig(stream=params["log_file"].value, filemode="a", level=logging.INFO)
logging.info(sys.version_info)

import collections
import itertools

import numpy as np
import pandas as pd

import scanpy as sc
from anndata import AnnData

sc.logging.logfile = params["log_file"].value  # for scanpy's own logging

import scrublet as scr
logging.info(scr.__file__)

Log config and settings

In [ ]:
# Reproducibility settings
import random

seed = 42
random.seed(seed)
logging.info("random seed {}".format(seed))
hash_seed = os.environ.get("PYTHONHASHSEED")
logging.info(f"PYTHONHASHSEED= {hash_seed}")
if hash_seed != 0:
    logging.warning(
        "Set PYTHONHASHSEED environmnetal variable to 0 for reproducibility"
    )

In [ ]:
# scanpy setttings
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

In [ ]:
# Increase plot resolution
sc.settings.set_figure_params(dpi=80)

## Read the input data and filter doublets

In [ ]:
input_dir = sorted(
    os.listdir(params["input_dir"].value)
)  # sort to ensure order is OS independent

adata_list = []
for each_dir in input_dir:
    adata_tmp = sc.read_10x_mtx(
        os.path.join(
            params["input_dir"].value, each_dir
        ),  # the directory with the `.mtx` file
        var_names="gene_symbols",  # use gene symbols for the variable names (variables-axis index)
        cache=True,
    )  # write a cache file for faster subsequent reading # will this work with list?

    # Check doublets in each 'batch' seperately
    scrub = scr.Scrublet(adata_tmp.X)
    (
        adata_tmp.obs["doublet_scores"],
        adata_tmp.obs["predicted_doublets"],
    ) = scrub.scrub_doublets()
    scrub.plot_histogram()
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
    scrub.plot_embedding('UMAP', order_points=True)
    if params["doublet_rem"].value:
        # Actually do the filtering
        adata_tmp = adata_tmp[adata_tmp.obs["predicted_doublets"] == False, :]

    adata_list.append(adata_tmp)

Concatenate the different batches (note you may want to name the batch categories)

In [ ]:
# Use an older version of AnnData since the newer versions 
# have this bug that makes concatentaion slow
# https://github.com/theislab/anndata/issues/303
adata_merged = AnnData.concatenate(
    *adata_list, join="outer"
)  # outer means union rather than intersection,
# Note will fill with 0s
adata_merged

In [ ]:
# Check .obs look normal
display(adata_merged.obs.head(2))
adata_merged.obs.tail(2)

In [ ]:
# Check the .var looks normal
display(adata_merged.var.head(2))
adata_merged.var.tail(2)

In [ ]:
adata_merged.var_names_make_unique()  # this is unnecessary if using 'gene_ids'

## Filtering and normalisation

Show those genes that yield the highest fraction of counts in each single cells, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata_merged, n_top=20)

Minimal filtering

In [ ]:
sc.pp.filter_cells(adata_merged, min_genes=params["min_genes"].value)
sc.pp.filter_genes(adata_merged, min_cells=params["min_cells"].value)

In [ ]:
mito_genes = adata_merged.var_names.str.startswith("MT-")
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata_merged.obs["percent_mito"] = (
    np.sum(adata_merged[:, mito_genes].X, axis=1).A1 / np.sum(adata_merged.X, axis=1).A1
)
# add the total counts per cell as observations-annotation to adata
adata_merged.obs["n_counts"] = adata_merged.X.sum(axis=1).A1

In [ ]:
sc.pl.violin(adata_merged, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

In [ ]:
# More optional filtering
if params["filter_n"].value:
    adata_merged = adata_merged[adata_merged.obs['n_genes'] < 4500, :]
    
if params["filter_mt"].value:
    adata_merged = adata_merged[adata_merged.obs['percent_mito'] < 0.05, :]

Normalize data

In [ ]:
#after normalization, each observation (cell) has a total count equal to 
# the median of total counts for observations (cells) before normalization"
sc.pp.normalize_total(adata_merged,
                      exclude_highly_expressed=params["exclude_high"].value)

Log-transform the data

In [ ]:
sc.pp.log1p(adata_merged)

In [ ]:
logging.info(f'Total number of cells: {adata_merged.n_obs:d}\n'
             f'Total number of genes: {adata_merged.n_vars:d}')

Save the results

In [ ]:
adata_merged.write(params["output_file"].value, compression='gzip')

Other downstream analyses deliberately not included